In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, random
import matplotlib.dates as mdates

In [ ]:
def MakeDataFromText(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif len(line)==8:               # If it is a time stamp
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h) - 3, int(m), int(s))) # Minus 3 for Pacific Time
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

class Person:
    """
    Class to represent each individal poutfit being recorded during Deployment, made up of garments.
    """
    def __init__(
        self, wet:bool, Name:str, color:str,
        backneck:str,   back:str,        chest:str, 
        leftab:str,     rightab:str, 
        rightuparm:str, rightlowarm:str, 
        leftuparm:str,  leftlowarm:str, 
        beanie1:str,    beanie2:str,     beanie3:str, beanieR:str, beanieL:str
    ):
        self.Backneck    = backneck        # Tag ID's for the shirt
        self.Back        = back
        self.Chest       = chest
        self.Leftab      = leftab
        self.Rightab     = rightab
        self.Rightuparm  = rightuparm
        self.Rightlowarm = rightlowarm
        self.Leftuparm   = leftuparm
        self.Leftlowarm  = leftlowarm
        self.Beanie1     = beanie1         # Tag ID's for the beanies
        self.Beanie2     = beanie2
        self.Beanie3     = beanie3
        self.BeanieR     = beanieR
        self.BeanieL     = beanieL

        self.Name = Name                   # Soldier's Name

        self.ShirtMapping = {
            self.Backneck:    Name + " Back Neck",
            self.Back:        Name + " Back",
            self.Chest:       Name + " Chest",
            self.Leftab:      Name + " Left Abdomen",
            self.Rightab:     Name + " Right Abdomen",
            self.Rightuparm:  Name + " Right Upper Arm",
            self.Rightlowarm: Name + " Right Lower Arm",
            self.Leftlowarm:  Name + " Left Lower Arm",
            self.Leftuparm:   Name + " Left Upper Arm",
        }

        self.TagList = [
            self.Backneck,   self.Back, self.Chest,
            self.Leftab,     self.Rightab,
            self.Rightuparm, self.Rightlowarm,
            self.Leftuparm,  self.Leftlowarm,
            self.Beanie1,    self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL
        ]

        self.BeanieList = [self.Beanie1, self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL]
        self.Color = color

        self.wet = wet # Bool for wetness check
        if wet:
            self.BackColor = "#870101"
            self.BackNeckColor = "#ad4503"
            self.ChestColor = "#756a01"
            self.LeftAbColor = "#396102"
            self.RightAbColor = "#015c0b"
            self.LeftUpArmColor = "#01663f"
            self.RightUpArmColor = "#00a191"
            self.LeftLowArmColor = "#012173"
            self.RightLowArmColor = "#1e075e"
            
        else:
            self.BackColor = "#ff0d0d"
            self.BackNeckColor = "#ff7214"
            self.ChestColor = "#d1bc02"
            self.LeftAbColor = "#6bb803"
            self.RightAbColor = "#01bf16"
            self.LeftUpArmColor = "#03c278"
            self.RightUpArmColor = "#06b4cf"
            self.LeftLowArmColor = "#0443e0"
            self.RightLowArmColor = "#4c05e6"

In [ ]:
person6  = Person(True, "Pa. P.", "#de0514", "71BB", "7705", "B25E", "3247", "A9B5", "6C38", "7662", "A983", "098F", "B644", "7BCC", "B576", "70D1", "4D84");
person14 = Person(True, "Jo.",    "#9e06a1", "462C", "5B60", "7415", "6310", "1851", "616A", "5DAA", "6D28", "9991", "3B0E", "5374", "682C", "0A50", "1543");
person28 = Person(True, "Ga. C.", "#0909eb", "8977", "4EB3", "AA5E", "6626", "9ABE", "6CD0", "AB8E", "14AA", "6E94", "895D", "571F", "2D68", "8121", "198C");
person2  = Person(True, "De.",    "#31e823", "333B", "289B", "2473", "231D", "9879", "4067", "5FB6", "169E", "8D50", "74C4", "73DC", "DC4A", "884D", "1BA4");
person17 = Person(True, "Br. L.", "#e8c102", "6CC7", "8FB7", "799D", "6F8D", "6332", "5A1E", "92A5", "4A3D", "3EC9", "16AB", "A66C", "6162", "A686", "7E1B");

person15 = Person(True, "Ja.",    "#0c028f", "9C54", "9968", "65E1", "5E23", "ACB3", "7AD3", "99A1", "B19D", "43C6", "7DCB", "9D35", "6FD0", "34CE", "62E3");
person16 = Person(True, "Ma. C.", "#04822d", "959E", "1C5F", "5A59", "077A", "902A", "3B60", "8199", "4469", "813A", "B7A4", "5163", "4DA5", "8995", "7972");
person13 = Person(True, "Wi. D.", "#f11de9", "A033", "A0C0", "892F", "627F", "6BD9", "3DB0", "8C97", "4ECF", "A73C", "90AE", "78D3", "82D8", "312D", "2A29");
person32 = Person(True, "Se.",    "#b58e0d", "2990", "1EC3", "51E2", "3783", "AE5E", "4575", "0688", "A0B0", "1BA6", "3C3E", "1989", "7C2C", "42D7", "6129");
person19 = Person(True, "Ha.",    "#023952", "697F", "78A5", "5D0E", "7EC6", "AE59", "8158", "4A9B", "1D44", "2122", "5C02", "2D90", "6DAB", "3374", "B1AB");
person26 = Person(True, "Ka.",    "#1ab6f2", "4813", "2791", "1F7A", "6F4A", "2383", "2F66", "2E5C", "926A", "2E56", "158F", "5B06", "B28A", "73C6", "819C");

# person11 = Person(False, "Jackson Dry", "#3845ff", "2A1B", "238C", "731F", "9591", "5C98", "5F06", "4526", "461C", "5253", "B078", "3AC7", "566B", "65D4", "432B");
# person8  = Person(False, "Matt C. Dry", "#00ed4f", "1B6A", "0D42", "7AD4", "20AF", "493F", "404A", "6878", "1A3B", "546F", "194B", "3133", "A847", "0D50", "7A61");
# person10 = Person(False, "William D. Dry", "#ff3037", "366C", "A08B", "AC42", "9AC9", "B53F", "76A4", "5E76", "68AE", "41D6", "803F", "5757", "9E54", "----", "----");
# person31 = Person(False, "Sean Dry", "#fb24ff", "1CAE", "7764", "8F24", "AD3E", "8898", "413F", "097A", "18A8", "4258", "7F4E", "801E", "8651", "AAB4", "6CB5");
# person21 = Person(False, "Harold Dry", "#008dc9", "8CB4", "4115", "4D74", "4883", "4C9F", "ABB7", "7938", "390F", "21A5", "A697", "7C7D", "B689", "114D", "047A");
# person22 = Person(False, "Katherina Dry", "#ed8600", "4F0D", "90D5", "0874", "45CE", "6CAC", "3EB2", "A26F", "888C", "50C3", "3767", "664F", "36A4", "2759", "BB8B");

t1 = MakeDataFromText("MWTC/trevor_test_1.txt")
t2 = MakeDataFromText("MWTC/trevor_test_2.txt")
t3 = MakeDataFromText("MWTC/trevor_test_3.txt")
t4 = MakeDataFromText("MWTC/trevor_test_4.txt")
t5 = MakeDataFromText("MWTC/trevor_test_5.txt")
t6 = MakeDataFromText("MWTC/trevor_test_6.txt")
t7 = MakeDataFromText("MWTC/trevor_test_7.txt")
t8 = MakeDataFromText("MWTC/trevor_test_8.txt")
y1 = MakeDataFromText("MWTC/yoel_test_1.txt")
y2 = MakeDataFromText("MWTC/yoel_test_2.txt")
y3 = MakeDataFromText("MWTC/yoel_test_3.txt")
y4 = MakeDataFromText("MWTC/yoel_test_4.txt")

combined = dict()
for d in [t1, t2, t3, t4, t5, t6, t7, t8, y1, y2, y3, y4]:
    for k,v in d.items():
        if k in combined: # if tag is already in dict
            for i in range(len(v[0])):
                if v[0][i] in combined[k]:
                    combined[k][v[0][i]].append(v[1][i])
                else:
                    combined[k][v[0][i]] = [v[1][i]]
        else:             # if tag is not in dict
            combined[k] = dict()
            for i in range(len(v[0])):
                combined[k][v[0][i]] = [v[1][i]]
for k,v in combined.items():
    for k2,v2 in v.items():
        combined[k][k2] = sum(v2)/len(v2)

In [ ]:
# WET-ONLY PARTICIPANTS

d6_x, d6_y = [], []
for k,v in combined.items():
    if k[-4:]==person6.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 7, 47, 0) and i[0]<datetime.datetime(1970, 2, 1, 8, 21, 0):
                d6_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 8, 1, 0)))
                d6_y.append(i[1])

d14_x, d14_y = [], []
for k,v in combined.items():
    if k[-4:]==person14.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 6, 45, 0) and i[0]<datetime.datetime(1970, 2, 1, 7, 15, 0):
                d14_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 6, 55, 30)))
                d14_y.append(i[1])

d28_x, d28_y = [], []
for k,v in combined.items():
    if k[-4:]==person28.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 8, 2, 0) and i[0]<datetime.datetime(1970, 2, 1, 8, 40, 0):
                d28_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 8, 15, 0)))
                d28_y.append(i[1])

d17_x, d17_y = [], []
for k,v in combined.items():
    if k[-4:]==person17.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 7, 5, 0) and i[0]<datetime.datetime(1970, 2, 1, 7, 40, 0):
                d17_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 7, 14, 0)))
                d17_y.append(i[1])

d2_x, d2_y = [], []
for k,v in combined.items():
    if k[-4:]==person2.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 7, 20, 15) and i[0]<datetime.datetime(1970, 2, 1, 7, 57, 30):
                d2_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 7, 33, 0)))
                d2_y.append(i[1])

In [ ]:
# WET AND DRY PARTICIPANTS

d15_x, d15_y = [], []
for k,v in combined.items():
    if k[-4:]==person15.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 7, 51, 10) and i[0]<datetime.datetime(1970, 2, 1, 8, 18, 45):
                d15_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 8, 1, 0)))
                d15_y.append(i[1])

d16_x, d16_y = [], []
for k,v in combined.items():
    if k[-4:]==person16.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 7, 25, 15) and i[0]<datetime.datetime(1970, 2, 1, 7, 52, 0):
                d16_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 7, 33, 0)))
                d16_y.append(i[1])

d13_x, d13_y = [], []
for k,v in combined.items():
    if k[-4:]==person13.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 7, 30, 10) and i[0]<datetime.datetime(1970, 2, 1, 8, 5, 0):
                d13_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 7, 47, 30)))
                d13_y.append(i[1])

d32_x, d32_y = [], []
for k,v in combined.items():
    if k[-4:]==person32.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 7, 5, 15) and i[0]<datetime.datetime(1970, 2, 1, 7, 32, 0):
                d32_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 7, 13, 0)))
                d32_y.append(i[1])

d19_x, d19_y = [], []
for k,v in combined.items():
    if k[-4:]==person19.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 8, 2, 30) and i[0]<datetime.datetime(1970, 2, 1, 8, 32, 15):
                d19_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 8, 13, 30)))
                d19_y.append(i[1])

d26_x, d26_y = [], []
for k,v in combined.items():
    if k[-4:]==person26.Backneck:
        s = sorted( v.items() )
        for i in s:
            if i[0]>datetime.datetime(1970, 2, 1, 6, 40, 30) and i[0]<datetime.datetime(1970, 2, 1, 7, 12, 0):
                d26_x.append(datetime.datetime(1970, 2, 1, 0, 0, 0) + (i[0] - datetime.datetime(1970, 2, 1, 6, 55, 20)))
                d26_y.append(i[1])

In [ ]:
# PLOTTING CODE
fig, ax = plt.subplots(figsize=(12, 6))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

# Plotting Wet Only Participants
ax.plot(d6_x, d6_y,   marker=".", color=person6.Color,  label=person6.Name)
ax.plot(d14_x, d14_y, marker=".", color=person14.Color, label=person14.Name)
ax.plot(d28_x, d28_y, marker=".", color=person28.Color, label=person28.Name)
ax.plot(d17_x, d17_y, marker=".", color=person17.Color, label=person17.Name)
ax.plot(d2_x, d2_y,   marker=".", color=person2.Color,  label=person2.Name)

# Plotting Wet and Dry Participants
ax.plot(d15_x, d15_y, marker=".", color=person15.Color, label=person15.Name)
ax.plot(d16_x, d16_y, marker=".", color=person16.Color, label=person16.Name)
ax.plot(d13_x, d13_y, marker=".", color=person13.Color, label=person13.Name)
ax.plot(d32_x, d32_y, marker=".", color=person32.Color, label=person32.Name)
ax.plot(d19_x, d19_y, marker=".", color=person19.Color, label=person19.Name)
ax.plot(d26_x, d26_y, marker=".", color=person26.Color, label=person26.Name)


times = [
    datetime.datetime(1970, 1, 31, 23, 45, 0) + datetime.timedelta(minutes=i) for i in range(40)
]
xticks = [
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", 
    "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", 
    "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", 
    "34", "35", "36", "37", "38", "39",
]

ax.set_xticks(times, xticks)
ax.set_yticks(
    [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30],
    ["2", "4", "6", "8", "10", "12", "14", "16", "18", "20", "22", "24", "26", "28", "30"]
)
ax.set_title("Back Neck Temperatures over Time for All Participants", fontsize=15)
ax.set_ylabel("Temperature (°C)", fontsize=15)
ax.set_xlabel("Minutes Elapsed", fontsize=15)
ax.set_ylim(4.0, 30.0)
ax.legend()